# Homework 7
## Deep Learning Technologies
### Shruti Badrinarayanan - 016768141

## Step 1: Load the Wikipedia GLoVE Word2Vec

In [ ]:
!pip install gensim
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip -d glove

In [14]:
from gensim.scripts.glove2word2vec import glove2word2vec

glove_input_file = 'glove/glove.6B.50d.txt'
word2vec_output_file = 'glove/glove.6B.50d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

from gensim.models import KeyedVectors

# Load the converted model
model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

<ipython-input-14-968fe8421a85>:5: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file, word2vec_output_file)


## Step 2: Show how similar are these words:

                 Man and Woman

                 Chair and Throne

                 water and baby

In [15]:
# Check similarity between different pairs of words
words_pairs = [('man', 'woman'), ('chair', 'throne'), ('water', 'baby')]
for word1, word2 in words_pairs:
    print(f"Similarity between {word1} and {word2}: {model.similarity(word1, word2)}")

Similarity between man and woman: 0.8860337734222412
Similarity between chair and throne: 0.27968090772628784
Similarity between water and baby: 0.40810367465019226


## Step 3: Using these provide analogies for the following:

             _____ is to King as Woman is to Man.

             _____ is to Princess as Man is to Woman.

             _____ is to a woman as a child is to an adult.

In [16]:
# Define the analogy tasks
analogy_tasks = [
    ('king', 'man', 'woman'),
    ('princess', 'woman', 'man'),
    ('adult', 'child', 'woman')
]

# Solve each analogy
for word1, word2, word3 in analogy_tasks:
      # Predict the most similar word
      predicted = model.most_similar(positive=[word1, word3], negative=[word2], topn=1)
      print(f"{predicted[0][0]} is to {word1} as {word3} is to {word2}.")

queen is to king as woman is to man.
prince is to princess as man is to woman.
male is to adult as woman is to child.


## Step 4: Apply Naive-Bayes Classifier on the Spam-Ham dataset shown in the demo.

In [17]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
from google.colab import drive
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from gensim.models.word2vec import Word2Vec
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import scale
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [18]:
# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/Deep Learning/Assignments/Assignment 7/spam.csv', encoding='latin-1')

# Only keep the necessary columns and rename them for convenience
data = data[['v1', 'v2']]
data.columns = ['label', 'message']

# Define a function for preprocessing text
def preprocess_text(sms):
    # Remove punctuation
    sms = "".join([char for char in sms if char not in string.punctuation])
    # Convert text to lowercase
    sms = sms.lower()
    # Remove stopwords
    sms_tokens = word_tokenize(sms)
    sms_clean = [word for word in sms_tokens if word not in stopwords.words('english')]
    return " ".join(sms_clean)

# Apply preprocessing to the messages
data['message_clean'] = data['message'].apply(preprocess_text)

# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data['message_clean'], data['label'], test_size=0.2, random_state=42)

# Feature Extraction using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Train a Naive Bayes Classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, y_train)

# Make predictions
y_pred_train = nb_classifier.predict(X_train_tfidf)
y_pred_test = nb_classifier.predict(X_test_tfidf)

# Evaluate the model
conf_matrix_train = confusion_matrix(y_train, y_pred_train)
conf_matrix_test = confusion_matrix(y_test, y_pred_test)
accuracy_train = accuracy_score(y_train, y_pred_train)
accuracy_test = accuracy_score(y_test, y_pred_test)

(conf_matrix_train, accuracy_train, conf_matrix_test, accuracy_test)

(array([[3860,    0],
        [  80,  517]]),
 0.9820507067534215,
 array([[965,   0],
        [ 33, 117]]),
 0.9704035874439462)

In [19]:
print("Training Set Results:\n")
print("Confusion Matrix:")
print(conf_matrix_train)
print(f"Accuracy: {accuracy_train * 100:.2f}%")

Training Set Results:

Confusion Matrix:
[[3860    0]
 [  80  517]]
Accuracy: 98.21%


In [20]:
print("\nTest Set Results:\n")
print("Confusion Matrix:")
print(conf_matrix_test)
print(f"Accuracy: {accuracy_test * 100:.2f}%")


Test Set Results:

Confusion Matrix:
[[965   0]
 [ 33 117]]
Accuracy: 97.04%
